<a href="https://colab.research.google.com/github/rideal-startup/rideal-ai/blob/master/notebooks/Federated_Learning_Rideal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated learning - Rideal

PySyft is a Python library for secure and private Deep Learning. PySyft decouples private data from model training, using Federated Learning, Differential Privacy, and Multi-Party Computation (MPC) within the main Deep Learning frameworks like PyTorch.

We download and install the library.

In [0]:
!pip install syft

Import the modules that we are going to use throughout the training.

In [0]:
from typing import Tuple
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Data Ingestion

Get the preprocessed dataset.

The dataset contains the following mobile sensors:
- Accelerometer
- Gyroscope
- Sound


In [0]:
df = pd.read_csv('/gdrive/My Drive/dataset-w5.csv')
x = df.drop(['target', 'time'], axis='columns').values.astype('float32')
y = df['target'].values 

In [0]:
x.shape, y.shape

((5893, 12), (5893,))

We create a mapping to convert the data labels to numeric indices. This is what we call a Label Encoder

In [0]:
IDX_2_LABEL = ['Still', 'Car', 'Train', 'Bus', 'Walking']
LABEL_2_IDX = {l: i for i, l in enumerate(IDX_2_LABEL)}

In [0]:
# Convert categorical labels to numbers
y = np.array([LABEL_2_IDX[l] for l in y])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.85)

train_mean = x_train.mean(axis=0)
train_std = x_train.std(axis=0)

x_train = (x_train - train_mean) / train_std
x_test = (x_test - train_mean) / train_std

Set PySyft hook to PyTorch

In [0]:
import syft as sy
hook = sy.TorchHook(torch)

## Simulate 4 different workers

In [0]:
guillem = sy.VirtualWorker(hook, id="guillem")
jordi = sy.VirtualWorker(hook, id="jordi")
atto = sy.VirtualWorker(hook, id="atto")
marcel = sy.VirtualWorker(hook, id="marcel")

Distribute the data over the workers


In [0]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

rand_idx = torch.randperm(x_train.shape[0]).long()
splits = torch.split(rand_idx, x.shape[0] // 4, dim=0)[:4]

guillem_data, guillem_target = x_train[splits[0]], y_train[splits[0]]
jordi_data, jordi_target = x_train[splits[1]], y_train[splits[1]]
atto_data, atto_target = x_train[splits[2]], y_train[splits[2]]
marcel_data, marcel_target = x_train[splits[3]], y_train[splits[3]]

In [0]:
guillem_data = guillem_data.send(guillem)
guillem_target = guillem_target.send(guillem)

jordi_data = jordi_data.send(jordi)
jordi_target = jordi_target.send(jordi)

atto_data = atto_data.send(atto)
atto_target = atto_target.send(atto)

marcel_data = marcel_data.send(marcel)
marcel_target = marcel_target.send(marcel)

Now the Tensors are PointerTensor because they are living in an another machine.

In [0]:
guillem_data

(Wrapper)>[PointerTensor | me:16966620659 -> guillem:6820356034]

In [0]:
# Organize the distributed datasets
datasets = [(guillem_data, guillem_target),
            (jordi_data, jordi_target),
            (atto_data, atto_target),
            (marcel_data, marcel_target)]

## Defining the model

We are going to implement a Voting Ensamble of Dense classifiers.

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [0]:
class Classifier(nn.Module):

  def __init__(self, n_features, n_labels):
    super(Classifier, self).__init__()
    self.linear1 = nn.Linear(n_features, 1024)

    self.linear2 = nn.Linear(1024, 512)

    self.linear3 = nn.Linear(512, 256)
    self.dropout3 = nn.Dropout(.5)

    self.linear4 = nn.Linear(256, 128)
    self.dropout4 = nn.Dropout(.5)

    self.clf = nn.Linear(128, n_labels)

  def forward(self, x):
    x = F.relu(self.linear1(x))
    
    x = F.relu(self.linear2(x))

    x = F.relu(self.linear3(x))
    x = self.dropout3(x)
    
    x = F.relu(self.linear4(x))
    x = self.dropout4(x)

    x = self.clf(x)

    return x

In [0]:
lr = 1e-2

def create_optimizers(m):
  """Create an optimizer for every worker"""
  return {
      guillem.id: torch.optim.SGD(params=m.parameters(), lr=lr),
      jordi.id: torch.optim.SGD(params=m.parameters(), lr=lr),
      marcel.id: torch.optim.SGD(params=m.parameters(), lr=lr),
      atto.id: torch.optim.SGD(params=m.parameters(), lr=lr),
  }

models = [Classifier(12, len(IDX_2_LABEL)) for _ in range(N_MODELS)]
for m in models:
  m.to(device)
optimizers = [create_optimizers(m) for m in models]

## Training

In [0]:
#@title Training hyperparameters

NUM_EPOCHS =  15#@param {type: "number"}
STEPS_PER_EPOCH = 256 #@param {type: "number"}
BATCH_SIZE = 32 #@param {type: "number"}
N_MODELS = 3 #@param {type: "number"}

In [0]:
def train_single_epoch(model: nn.Module, 
                       optimizer: dict,
                       epoch: int):
  losses = []
  model.train()

  for iter in range(STEPS_PER_EPOCH):
    for data, target in datasets:
          
      model.send(data.location)

      optimizer[data.location.id].zero_grad()

      predictions = model(data.to(device))

      loss = F.cross_entropy(predictions, target.view(-1).to(device))
      loss.backward()

      optimizer[data.location.id].step()
      
      model.get()
      losses.append(loss.get().item())
  
  loss_mean = sum(losses) / len(losses)
  print(f'Epoch[{epoch}] training loss: {loss_mean}')


def train_model(model: nn.Module, 
                optimizer: torch.optim.Optimizer):
  
  for epoch in range(NUM_EPOCHS):
    train_single_epoch(model, optimizer, epoch)


for i in range(N_MODELS):
  print('----- Training model {} -----'.format(i))
  train_model(models[i], optimizers[i])
  torch.save(models[i].state_dict(), '/gdrive/My Drive/model_{}.pt'.format(i))

----- Training model 0 -----
Epoch[0] training loss: 1.3471073576947674
Epoch[1] training loss: 1.0932938750484027
Epoch[2] training loss: 0.9873493363847956
Epoch[3] training loss: 0.910288873128593
Epoch[4] training loss: 0.8422362464480102
Epoch[5] training loss: 0.7896337371785194
Epoch[6] training loss: 0.7490539705031551
Epoch[7] training loss: 0.7130193212069571
Epoch[8] training loss: 0.6804021842544898
Epoch[9] training loss: 0.6503270207322203
----- Training model 1 -----
Epoch[0] training loss: 1.3397509147180244
Epoch[1] training loss: 1.0783673717523925
Epoch[2] training loss: 0.9754909795592539
Epoch[3] training loss: 0.8968368303612806
Epoch[4] training loss: 0.8309920413303189
Epoch[5] training loss: 0.7822642175597139
Epoch[6] training loss: 0.7424696591915563
Epoch[7] training loss: 0.7075948596466333
Epoch[8] training loss: 0.6762245220597833
Epoch[9] training loss: 0.6478326536598615
----- Training model 2 -----
Epoch[0] training loss: 1.3496386968763545
Epoch[1] tr

In [0]:
def save(models):
  torch.save([m.state_dict() for m in models],
            '/gdrive/My Drive/tmd_detection_ensamble.py')

In [0]:
@torch.no_grad()
def predict(x, models):
  predictions = torch.stack([m(x.to(device)).argmax(-1) 
                             for m in models])
  predictions = predictions.permute(1, 0)
  voting_preds = []
  for preds in predictions:
    counts = Counter(preds.tolist())
    most_common = counts.most_common()
    if len(most_common) == 1:
      voting_preds.append(most_common[0][0])
    else:
      values = torch.FloatTensor([o[1] for o in most_common])
      classes = [o[0] for o in most_common]
      probs = torch.softmax(values, dim=0).numpy()
      voting_preds.append(np.random.choice(classes, p=probs, size=(1,))[0])
  
  return torch.LongTensor(voting_preds)


def report_results():
  x = torch.from_numpy(x_test)
  y = torch.from_numpy(y_test)

  preds = predict(x, models)
  
  y_trues = y.view(-1).cpu().numpy()
  y_preds = preds.cpu().numpy()

  print(metrics.classification_report(y_trues, y_preds))

In [0]:
report_results()

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       176
           1       0.57      0.60      0.59       173
           2       0.69      0.65      0.67       168
           3       0.72      0.69      0.70       179
           4       0.93      0.88      0.90       188

    accuracy                           0.74       884
   macro avg       0.74      0.74      0.74       884
weighted avg       0.74      0.74      0.74       884

